# Rental External Data Pre-processing 

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('../data/landing/Moving annual rent by suburb - March quarter 2023.xlsx', sheet_name='All properties')

In [4]:
def hist_rental_price(df):
    
    """
    Process historical rental price data from a DataFrame and return a summary DataFrame.

    Parameters:
    df: The input DataFrame containing historical rental price data.

    Returns:
    DataFrame: A summary DataFrame with columns for suburb names, years, and median rental prices.
    """

    # Remove the last row, which may contain summary information
    df = df.iloc[:-1]

    # Extract suburb names starting from the third row (index 2) and second column (index 1)
    suburb_names = df.iloc[2:, 1].values

    # Make an explicit copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Add years to the first row using the year_addition function
    df.iloc[0, :] = year_addition(df)

    # Create a filtered DataFrame, excluding the header row and selecting relevant columns
    filtered_df = df.loc[1:].copy()
    filtered_df.columns = df.iloc[0, :]
    name = filtered_df.iloc[:, 1]
    filtered_df = filtered_df.iloc[:, 2:]  # This selects all rows and all columns starting from the second column

    # Initialize a new DataFrame to store the results
    result_df = pd.DataFrame()
    year_columns = df.iloc[0, 2:].tolist()
    year_columns = list(set(year_columns))

    # Iterate through the year columns
    for year in year_columns:
        year_result = median_mean(filtered_df[year], year)
        result_df = pd.concat([result_df, year_result], axis=1)

    # Reset the index
    result_df.reset_index(drop=True, inplace=True)

    # Insert the name of each suburb as the first column
    result_df.insert(0, 'Name', suburb_names)

    return result_df

In [5]:
def year_addition(df):
    
    """
    Extract and transform years from a DataFrame and return them as a list.

    Parameters:
    df: The input DataFrame containing year information in its columns.

    Returns:
    list: A list containing the extracted and transformed years.
    """

    # Extract the initial list of years from the first row of the DataFrame
    year_list = df.iloc[0, :].tolist()

    # Initialize a list to store the transformed years, including the first two elements
    result_list = year_list[:2]

    # Iterate through the remaining year values
    for item in year_list[2:]:
        if isinstance(item, str):
            # Extract the year from the string and convert it to an integer
            year = int((item).split()[-1])
            result_list.append(year)
        else:
            # If the item is not a string, append the previously extracted year
            result_list.append(year)

    return result_list

In [6]:
def median_mean(df, year):
    
    """
    Calculate the mean of median values for a specific year from a DataFrame.

    Parameters:
    df: The DataFrame containing median values.
    year: The specific year for which the median mean should be calculated.

    Returns:
    DataFrame: A DataFrame containing the mean of median values for the specified year.
    """

    # Replace hyphens with NaN for consistency in data
    df = df.replace('-', np.nan)

    # Select only the rows containing medians (excluding the first two rows)
    median_mask = (df.iloc[0] == "Median")
    median_rows = df.loc[:, median_mask]

    # Exclude the header rows and convert NaN to 0 for numerical calculations
    median_num = median_rows.iloc[1:, :]
    median_num = median_num.fillna(0).astype(int)

    # Calculate the mean of medians along the rows
    median_mean = median_num.mean(axis=1)

    # Round the mean values to 2 decimal places
    median_mean = median_mean.round(2)

    # Create a new DataFrame with the year and the median mean as a single column
    result_df = pd.DataFrame({f"{year}": median_mean})

    return result_df


### Process historical rental price data using the hist_rental_price function

In [7]:
historial_data = hist_rental_price(df)
historial_data.head()

,Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Albert Park-Middle Park-West St Kilda,266.25,281.25,300.00,295.0,300.00,302.50,317.00,362.5,389.50,...,472.50,491.25,515.00,527.75,562.50,568.00,570.00,498.75,510.00,545.0
1,Armadale,203.75,222.50,231.00,236.0,238.75,243.25,261.25,295.0,331.25,...,393.75,399.75,402.50,445.00,487.50,481.25,498.75,433.75,447.50,490.0
2,Carlton North,263.75,276.25,290.00,290.0,299.50,301.25,322.75,345.0,388.75,...,515.00,530.00,527.50,545.00,566.25,579.25,588.75,577.50,593.75,620.0
3,Carlton-Parkville,260.00,263.75,271.67,281.2,282.50,280.25,275.00,279.5,290.00,...,351.75,365.50,373.25,398.00,417.00,431.25,399.75,335.00,329.75,400.0
4,CBD-St Kilda Rd,320.00,320.00,320.00,308.0,300.00,304.00,316.25,350.0,384.75,...,435.00,441.25,452.50,480.00,493.75,505.00,471.25,362.25,406.00,500.0


In [8]:
historial_data.to_csv("../data/raw/historical_rental_price_by_suburb.csv", index=False)